In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Data preprocessing
# Augmentation

In [ ]:

img_height, img_width = 224, 224

# Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
])

# Simple CNN Model

In [ ]:

def create_cnn_model(num_classes):
    model = models.Sequential([
        data_augmentation,
        layers.Rescaling(1./255),
        
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(),
        
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Create model

In [ ]:

model = create_cnn_model(len(class_names))



# Compile the model

In [ ]:

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Train

In [ ]:

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10  # Start small
)




# Save model

In [ ]:
# Save model
model.save('../models/baseline_cnn.h5')

# Plot results

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Model Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Model Loss')
plt.savefig('../results/baseline_training.png')

# Use pre-trained MobileNetV2

In [ ]:
from tensorflow.keras.applications import MobileNetV2


base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base





# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=10)
model.save('../models/baseline_mobilenet.h5')

# Adding custom head

In [ ]:
# Add custom head
model = models.Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train

In [ ]:


history = model.fit(train_ds, validation_data=val_ds, epochs=10)
model.save('../models/baseline_mobilenet.h5')